In [ ]:
!kaggle datasets download -d dskcproject/deepfake-frames

Dataset URL: https://www.kaggle.com/datasets/dskcproject/deepfake-frames
License(s): CC0-1.0
 96% 471M/493M [00:05<00:00, 122MB/s]
100% 493M/493M [00:05<00:00, 98.4MB/s]


In [ ]:
!unzip -qq '/content/deepfake-frames.zip'

replace content/Dataset/Train/Fake/fake-aaagqkcdis.mp4/Frame1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

val_datagen = ImageDataGenerator(rescale = 1./255)

train_set = train_datagen.flow_from_directory(
                                              '/content/content/Dataset/Train',
                                               target_size = (224, 224),
                                               batch_size = 64,
                                               class_mode = 'binary')

val_set = val_datagen.flow_from_directory(
                                            '/content/content/Dataset/Valid',
                                            target_size = (224, 224),
                                            batch_size = 64,
                                            class_mode = 'binary')


Found 173760 images belonging to 2 classes.
Found 28736 images belonging to 2 classes.


In [ ]:
!pip install keras-tuner

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, LSTM, Reshape, Input
from tensorflow.keras.applications import MobileNet, InceptionV3, ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from kerastuner import HyperParameters

def build_model(hp):
    # Input Layer for Images
    input_shape = (224, 224, 3)  # Image shape for 224x224 RGB images
    inputs = Input(shape=input_shape)

    # Select one of the pre-trained models (MobileNet, InceptionV3, or ResNet50)
    model_choice = hp.Choice('model_choice', values=['MobileNet', 'InceptionV3', 'ResNet50'])

    # Base Models - MobileNet, InceptionV3, ResNet50 (use pre-trained weights)
    if model_choice == 'MobileNet':
        base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif model_choice == 'InceptionV3':
        base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif model_choice == 'ResNet50':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    # Freeze the layers of the base model
    base_model.trainable = False

    # Add the base model to the inputs
    x = base_model(inputs)

    # Reshape the CNN output to make it compatible with LSTM (timesteps, features)
    # The CNN output has shape (batch_size, height, width, channels), which we can reshape.
    # We will treat height as the number of timesteps and width * channels as the number of features.
    x = Reshape((-1, x.shape[-1]))(x)  # Reshape to (batch_size, timesteps, features)

    # Add LSTM layer for temporal feature learning
    lstm_units = hp.Int('lstm_units', min_value=64, max_value=512, step=64)
    lstm_layer = LSTM(lstm_units, return_sequences=False)(x)

    # Dense Layers for final classification
    dense_units_1 = hp.Int('dense_units_1', min_value=8, max_value=264, step=8)
    dense_units_2 = hp.Int('dense_units_2', min_value=8, max_value=264, step=8)

    x = Dense(dense_units_1, activation='relu')(lstm_layer)
    x = Dense(dense_units_2, activation='relu')(x)
    output = Dense(1, activation='sigmoid')(x)

    # Final Model
    model = Model(inputs=inputs, outputs=output)

    # Compile the Model
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

    return model


In [ ]:
model = build_model(HyperParameters())
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ mobilenet_1.00_224 (Functional)      │ (None, 7, 7, 1024)          │       3,228,864 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 49, 1024)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 64)                  │         278,784 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 8)                   │             520 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 8)                   │              72 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │               9 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,508,249 (13.38 MB)

 Trainable params: 279,385 (1.07 MB)

 Non-trainable params: 3,228,864 (12.32 MB)

In [ ]:
from keras_tuner import Hyperband

# Initialize Keras Tuner
tuner = Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=10,
    factor=3,
    directory='my_dir',
    project_name='image_classification_tuning'
)

# Perform hyperparameter search using ImageDataGenerator
tuner.search(train_set, epochs=10, validation_data=val_set)

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Train the best model on the full dataset (already done by the tuner, but you can re-train if needed)
best_model.fit(train_set, epochs=10, validation_data=val_set)

# Save the trained model
best_model.save('best_classification_model.h5')



Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
MobileNet         |MobileNet         |model_choice
192               |192               |lstm_units
88                |88                |dense_units_1
112               |112               |dense_units_2
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
0                 |0                 |tuner/round



/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/2
2715/2715 ━━━━━━━━━━━━━━━━━━━━ 2100s 771ms/step - accuracy: 0.7301 - loss: 0.5257 - val_accuracy: 0.7177 - val_loss: 0.5979
Epoch 2/2
2555/2715 ━━━━━━━━━━━━━━━━━━━━ 1:56 728ms/step - accuracy: 0.8382 - loss: 0.3505

In [ ]:
train_set = train_datagen.flow_from_directory(
                                              '/content/content/Dataset/Train',
                                               target_size = (224, 224),
                                               batch_size = 64,
                                               class_mode = 'binary')

In [ ]:
from sklearn.metrics import classification_report
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model('best_classification_model.h5')

# Get predictions on the training set and new test set
train_predictions = model.predict(train_set)
test_predictions = model.predict(new_test_set)

# Convert predictions from probabilities to class labels (0 or 1 for binary classification)
train_predictions = (train_predictions > 0.5).astype(int)
test_predictions = (test_predictions > 0.5).astype(int)

# Evaluate the classification report (precision, recall, f1-score)
train_labels = train_set.labels  # Ground truth labels for train set
test_labels = new_test_set.labels  # Ground truth labels for test set

# Print the classification report for training and test data
print("Classification Report for Train Data:")
print(classification_report(train_labels, train_predictions))

print("Classification Report for Test Data:")
print(classification_report(test_labels, test_predictions))
